<a href="https://colab.research.google.com/github/DavidCarricondo/selenium-NLP/blob/master/RNN_for_reviews_polarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt


Create a function that plots the metrics of the model:


In [0]:
def plot_metric(history, metric):
  plt.plot(history[metric])
  plt.plot(history['val_'+metric, ''])
  plt.xlabel('Epochs')
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

LOAD THE DATASET

In [6]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteZK1XKB/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteZK1XKB/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteZK1XKB/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [44]:
#Look at the data
[print(e) for i,e in enumerate(dataset['test']) if i < 2 ]
  

(<tf.Tensor: shape=(283,), dtype=int64, numpy=
array([ 173,   29,  185,   13,  115, 1956, 8044,    3,  398, 1261, 5497,
        423,    2,   15,   18, 4096, 3958,  637, 2657,  552, 2893, 4926,
       2314, 1673, 4587,  137,   23, 4872, 5345,    2, 7996, 7277, 8004,
       8012,  137,   23,  972, 5346, 5977, 1365, 8051,    2, 7998, 3780,
       1947, 1747, 4411, 3743,    3, 5064, 7961,    7,   13, 2671, 4556,
       3449, 1678,  572, 8037, 7968,    8,  604, 1006, 4002, 2618, 7974,
       7994, 7974, 2893, 2618, 7974, 2475, 3912,    3, 4847, 2034, 2615,
         66, 7974, 3201,    2,    5,   20, 1668, 5475, 7961,    5,   20,
       4474,   20,  119,    6,    1, 2489, 1473, 1960,  323,    3,   12,
       1167, 7968,   21, 4101,   14,  366,  342,   12,  284,  552, 7999,
       4949, 3780, 3779, 3369, 8018,  625, 1979,    2,   78, 1259,    2,
         12,  109, 7968,   21,  129,   12, 4101,  236,   14, 1682,  230,
        392,    7, 4980, 1079, 1678,  572, 8037, 7968,    8, 1712,    9,
    

[None, None]

It seems like the data is already encoded...
Let's split it in training and testing


In [0]:
X_train = dataset['train']
X_test = dataset['test']

But the data comes with the encoder, which I can use for external data as well :)

In [25]:
#Is an encoder with a vocab_size of 8185 words
encoder = info.features['text'].encoder
encoder.vocab_size

8185

Let's try to decode the first two reviews:

In [52]:
for i, e in enumerate(X_train):
  if i < 2:
    print(encoder.decode(e[0]))
  else: break

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was cons

In [51]:
#Check the values of the first words of the first review
for inrev, rev in enumerate(X_train):
  if inrev==0:
    for inword, word in enumerate(rev[0]):
      if inword<20:
          print(f'{word} -----> {encoder.decode([word])}')
      else: break

62 -----> This 
18 -----> was 
41 -----> an 
604 -----> absolutely 
927 -----> terrible 
65 -----> movie
3 -----> . 
644 -----> Don
7968 -----> '
21 -----> t 
35 -----> be 
5096 -----> lur
36 -----> ed 
11 -----> in 
43 -----> by 
2948 -----> Christopher 
5240 -----> Walke
102 -----> n 
50 -----> or 
681 -----> Michael 


In [63]:
#I wonder if the encoder broadcast to tensor objects:
s = 'Here comes the sun'
s_encoded = encoder.encode(s)
print(type(s_encoded))
print(s_encoded)

<class 'list'>
[2844, 327, 1, 2365]


In [64]:
#Nope, but:
s_tensor = tf.convert_to_tensor(s_encoded)
print(type(s_tensor))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [65]:
#And this would be an input for the model (tuple of tensors or the type (input tensor, 0 tensor))
print((s_tensor, tf.convert_to_tensor(0)))

(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([2844,  327,    1, 2365], dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


Now, let's prepare the batches of inputs for the model shuffeling and then padding for the last one (adding zeros until the length is equal to the longest review)


In [0]:
BATCH_SIZE = 64

X_train = X_train.shuffle(buffer_size=10000)
X_train = X_train.padded_batch(BATCH_SIZE)

X_test = X_test.padded_batch(BATCH_SIZE)

Now we are ready to run a first model with keras API:

In [0]:
model = tf.keras.Sequential([
    #The embedding layer will have an input dimesion that is the vocabulary size, so the largest integer will not be larger than 8185
    tf.keras.layers.Embedding(input_dim = encoder.vocab_size, output_dim = 64, embeddings_initializer='uniform'),
    #Then we add a Long short-term memory layer with a bidirectional wrapper on it.
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64), merge_mode='concat'),
    #And then a dense layers
    tf.keras.layers.Dense(64, activation='relu'),
    #And the output dense layer with a linear activation (f(x) = x)
    tf.keras.layers.Dense(1)
])

And compile the model using a Binary corssentropy loss function and we'll update with an Adam optimizer on the accuracy.

In [0]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.Adam(1e-4),
              metrics = ['accuracy'])

In [88]:
history = model.fit(X_train, epochs=10,
                    validation_data=X_test, 
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 53s 135ms/step - loss: 0.6470 - accuracy: 0.5604 - val_loss: 0.4580 - val_accuracy: 0.7641
Epoch 2/10
391/391 [==============================] - 54s 138ms/step - loss: 0.3490 - accuracy: 0.8492 - val_loss: 0.3435 - val_accuracy: 0.8536
Epoch 3/10
391/391 [==============================] - 54s 137ms/step - loss: 0.2566 - accuracy: 0.8991 - val_loss: 0.3575 - val_accuracy: 0.8516
Epoch 4/10
391/391 [==============================] - 54s 138ms/step - loss: 0.2143 - accuracy: 0.9200 - val_loss: 0.3518 - val_accuracy: 0.8323
Epoch 5/10
391/391 [==============================] - 53s 137ms/step - loss: 0.1861 - accuracy: 0.9324 - val_loss: 0.3373 - val_accuracy: 0.8687
Epoch 6/10
391/391 [==============================] - 53s 137ms/step - loss: 0.1591 - accuracy: 0.9446 - val_loss: 0.3570 - val_accuracy: 0.8693
Epoch 7/10
391/391 [==============================] - 53s 137ms/step - loss: 0.1436 - accuracy: 0.9508 - val_loss: 0.3654 - val_ac

In [89]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 20s 52ms/step - loss: 0.4427 - accuracy: 0.8549
Test Loss: 0.4427180588245392
Test Accuracy: 0.8548799753189087


In [91]:
model.save('.')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ./assets


INFO:tensorflow:Assets written to: ./assets
